# Triplet Train

使用ipynb笔记本来对python代码进行逐步调试处理/训练

首先，读取数据。

In [4]:
import logging
import torch
import numpy as np
import tritrain
import dataset
import torch.optim as optim

from dissimilarity import calculate_cs_dissimilarity_matrix
from dataset import read_all_of_huaweicup,read_npy_of_file,npy_file_name_converter

from geodesic import construct_knn_graph,compute_shortest_paths_dijkstra,compute_shortest_paths_dijkstra_cugraph

# 尝试导入 cugraph 和 cudf，如果失败则使用 networkx
try:
    import cudf
    import cugraph
    GPU_AVAILABLE = True
except ImportError:
    GPU_AVAILABLE = False

## 2. 数据读入及预处理
使用 huawei 杯官方给的示例代码，导入数据。

准备数据集，首先定义有关函数。

需要注意，我们的数据集每一个对应着不同的扇区，这里先导一个，每个文件导出的时候也对应不同扇区，训练三个模型。

In [5]:
print("<<< Welcome to 2024 Wireless Algorithm Contest! >>>\n")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Round_idx = 1
File_idx = 3
bs_pos, tol_samp_num, anch_samp_num, port_num, ant_num, sc_num, anch_pos, H, d_cs, d_geo = read_all_of_huaweicup(Round_idx,File_idx)

# 初始化日志记录
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()


<<< Welcome to 2024 Wireless Algorithm Contest! >>>

Processing Round 1 Case 3
Loading configuration data file
Loading input position file
Loading input CSI data of Case 3
Loading Channel CSI succeed
Loading DCS data succeed
Loading GEO data succeed


## 3.网络训练

Triplet Neural Networks

![Triplet Neural Networks](/home/xmax/Desktop/huaweicup/pic/img.png)

### 3.1 Triplet Selection

输入为Dgeo矩阵，为两个采样点之间的接地线距离，输出为 f(i) , f(j) , f(k) ，其中 i 为2000个锚点， j 为反例，距离 i 很远的例子， k 为正例距离 i 很近的例子。
有一个参数Q，用来衡量近的点有多近，远的点有多远，是一个可调参数。
论文中，Q=0.02-0.2，此处为可调参数。

### 3.2 Embedded 网络


### 3.3 Triplet Loss


In [6]:
# 创建模型
embed_net = tritrain.TripletNet().to(device)
print("Selected device " + str(device))
# 初始化损失函数，给入参数 margin
margin = 1.0
criterion = tritrain.TripletLoss(margin).to(device)

H_real = H.real
H_imag = H.imag
H_combined = np.stack((H_real, H_imag), axis=2)
H_combined = H_combined.reshape(tol_samp_num, 4, ant_num, sc_num)

optimizer = optim.Adam(embed_net.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

# 训练网络
tritrain.train_triplet_network(embed_net, criterion, optimizer, scheduler , H_combined, d_cs, anch_pos, init_q=0.2)

# 保存网络实例
torch_net_file = dataset.tensor_file_name_converter(1,1)
torch.save(embed_net.state_dict(), torch_net_file)
print("Model saved as "+ torch_net_file)


Selected device cuda
使用cuda进行计算


Epoch 1/20:  16%|█▌        | 1009/6250 [00:22<01:54, 45.81it/s]

Epoch [1/20], Batch [1000], Q: 0.2000, Loss: 9.9535


Epoch 1/20:  32%|███▏      | 2009/6250 [00:44<01:32, 45.80it/s]

Epoch [1/20], Batch [2000], Q: 0.2000, Loss: 9.8830


...

Epoch [19/20], Batch [1000], Q: 0.0380, Loss: 5.9302


Epoch [19/20] Completed, Learning Rate: 0.000500


Epoch 20/20:  64%|██████▍   | 1005/1563 [00:36<00:20, 27.33it/s]

Epoch [20/20], Batch [1000], Q: 0.0290, Loss: 5.7368


Epoch [20/20] Completed, Learning Rate: 0.000250
Model saved as ./CompetitionData1/Round1NET11721481719.3856256.pth
